In [1]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
AIDEVS_API_KEY = os.environ["AIDEVS_API_KEY"]

In [3]:
client = OpenAI()

In [4]:
response = requests.post("https://tasks.aidevs.pl/token/scraper", json=dict(apikey=AIDEVS_API_KEY))
response_json = json.loads(response.text)
token = response_json["token"]

response = requests.get(f"https://tasks.aidevs.pl/task/{token}")
response_json = json.loads(response.text)

print(response_json)
article_url = response_json["input"]
question = response_json["question"]

{'code': 0, 'msg': 'Return answer for the question in POLISH language, based on provided article. Maximum length for the answer is 200 characters', 'input': 'https://tasks.aidevs.pl/text_pizza_history.txt', 'question': 'z którego roku pochodzi łaciński dokument, który pierwszy raz wspomina o pizzy? '}


In [6]:
headers={
    "Host": "tasks.aidevs.pl",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:123.0) Gecko/20100101 Firefox/123.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": b"1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "DNT": b"1",
    "Sec-GPC": b"1",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",

}

response = requests.get(article_url, headers=headers)
print(response)
with open("article.txt", mode="wb") as file:
    file.write(response.content)

<Response [200]>


In [7]:
with open("article.txt", mode="rb") as file:
    context_enc = file.read()
    context = context_enc.decode()
print(context)

Foods similar to pizza have been made since the Neolithic Age. Records of people adding other ingredients to bread to make it more flavorful can be found throughout ancient history. In the 6th century BC, the Persian soldiers of the Achaemenid Empire during the rule of Darius the Great baked flatbreads with cheese and dates on top of their battle shields and the ancient Greeks supplemented their bread with oils, herbs, and cheese. An early reference to a pizza-like food occurs in the Aeneid, when Celaeno, queen of the Harpies, foretells that the Trojans would not find peace until they are forced by hunger to eat their tables (Book III). In Book VII, Aeneas and his men are served a meal that includes round cakes (like pita bread) topped with cooked vegetables. When they eat the bread, they realize that these are the "tables" prophesied by Celaeno.
The first mention of the word "pizza" comes from a notarial document written in Latin and dating to May 997 AD from Gaeta, demanding a paymen

In [8]:
completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": context},
    {"role": "user", "content": question}
  ]
)

answer = str(completion.choices[0].message.content)
print(answer)

Pierwsze wspomnienie o pizzy pochodzi z łacińskiego dokumentu z maja 997 roku.


In [9]:
response = requests.post(f"https://tasks.aidevs.pl/answer/{token}", json=dict(answer=answer))
print(response.text)

{
    "code": 0,
    "msg": "OK",
    "note": "CORRECT"
}
